## Amazon Bedrock Knowledge Bases - 다양한 데이터 소스를 사용한 엔드 투 엔드 예제 ##
이 노트북은 Amazon Bedrock Knowledge Bases를 사용하여 RAG 애플리케이션을 구축하고 다양한 데이터 소스(S3, Confluence, Sharepoint, Salesforce 및 Web)에서 문서를 색인에 수집하는 엔드 투 엔드 예제의 샘플 코드를 제공합니다. 최대 5개의 데이터 소스를 추가할 수 있습니다.

노트북 설명
질문이 수신될 때 조회할 수 있도록 문서(일반적으로 여러 데이터 소스에 저장됨)를 Amazon OpenSearch Service Serverless(AOSS)와 같은 벡터 데이터베이스인 지식 베이스로 수집하는 데이터 파이프라인입니다.

다양한 데이터 소스(S3, Confluence, Sharepoint, Salesforce 및 Web)를 연결하여 문서를 지식 베이스에 로드합니다.
수집 - 지식 베이스는 문서를 더 작은 청크로 분할하고(선택한 전략에 따라), 임베딩을 생성하여 관련 벡터 저장소에 저장합니다.
<!-- data_ingestion.png -->
<img src="./images/data_ingestion.png" width=50% height=20% />

단계:
다양한 데이터 소스(S3, Confluence, Sharepoint, Salesforce 및 Web)에서 데이터에 액세스하고 OSS에 임베딩을 쓰는 데 필요한 정책이 포함된 Knowledge Base 실행 역할을 생성합니다.
빈 OpenSearch 서버리스 인덱스를 생성합니다.
사전 요구 사항:
S3의 경우, s3 버킷 생성(존재하지 않는 경우) 및 데이터 업로드
다른 데이터 소스의 경우 - 해당 [AWS 문서 페이지](https://docs.aws.amazon.com/bedrock/latest/userguide/data-source-connectors.html)의 사전 요구 사항 참조
지식 베이스 생성
지식 베이스 내에 데이터 소스 생성
각 데이터 소스에 대해 KB API를 사용하여 수집 작업을 시작하면 데이터 소스에서 데이터를 읽고, 청크로 분할하고, Amazon Titan Embeddings 모델을 사용하여 청크를 임베딩으로 변환한 다음 이러한 임베딩을 AOSS에 저장합니다. 이 모든 것이 데이터 파이프라인을 구축, 배포 및 관리할 필요 없이 이루어집니다.
데이터가 Bedrock Knowledge Base에서 사용 가능하게 되면 Amazon Bedrock에서 제공하는 Knowledge Base API를 사용하여 질문 답변 애플리케이션을 구축할 수 있습니다.

<div class="alert alert-block alert-info">
<b>참고:</b> Amazon Bedrock Console에서 다음 모델들에 대한 접근 권한을 활성화했는지 확인하십시오:
amazon.nova-micro-v1:0
Anthropic Claude 3 Sonnet
amazon.titan-text-express-v1
anthropic.claude-3-haiku-20240307-v1:0
Titan Text Embeddings V2 
<br> ------------------------------------------------------------------------------------------------------------------------------------------------------- </br>
"모든 셀 실행" 옵션을 사용하는 대신 노트북의 셀을 하나씩 실행해 주시기 바랍니다.
</div>

## 설정
이 노트북의 나머지 부분을 실행하기 전에, (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결하기 위해 아래 셀들을 실행해야 합니다.

In [ ]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))

# Print sys.path to verify
print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [ ]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

In [ ]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"

knowledge_base_name = f"bedrock-sample-knowledge-base-{suffix}"
knowledge_base_description = "Multi data source knowledge base."

bucket_name = f'{knowledge_base_name}-{account_id}'
intermediate_bucket_name = f'{knowledge_base_name}-intermediate-{account_id}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

In [ ]:
print(boto3.__version__)

### 이제 지식 베이스에 여러 가지 다양한 데이터 소스(S3, Confluence, Sharepoint, Salesforce, Web Crawler)를 추가할 수 있습니다. 이 노트북에서는 여러 가지 다양한 데이터 소스를 사용하여 지식 베이스 생성을 테스트해보겠습니다.

각 데이터 소스마다 서로 다른 사전 요구사항이 있을 수 있으므로, 자세한 내용은 AWS 문서를 참조하시기 바랍니다.

In [ ]:
# For this notebook, we'll create Knowledge Base with multiple data sources ( 1 S3 bucket, 1 confluence page, 1 Sharepoint site, 1 Salesforce site, 1 Web Crawler)

data_bucket_name = f'bedrock-kb-{suffix}-1' # replace it with your first bucket name.

## Below is a list of data sources including, 1 S3 buckets, 1 confluence, 1 Sharepoint, 1 Salesforce connectors
## Please uncomment the data sources that you want to add and update the placeholder values accordingly.

data_sources=[
                {"type": "S3", "bucket_name": data_bucket_name}, 
                
                # {"type": "CONFLUENCE", "hostUrl": "https://example.atlassian.net", "authType": "BASIC",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>"},

                # {"type": "SHAREPOINT", "tenantId": "888d0b57-69f1-4fb8-957f-e1f0bedf64de", "domain": "yourdomain",
                #   "authType": "OAUTH2_CLIENT_CREDENTIALS",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>",
                #  "siteUrls": ["https://yourdomain.sharepoint.com/sites/mysite"]
                # },

                # {"type": "SALESFORCE", "hostUrl": "https://company.salesforce.com/", "authType": "OAUTH2_CLIENT_CREDENTIALS",
                #  "credentialsSecretArn": f"arn:aws::secretsmanager:{region_name}:secret:<<your_secret_name>>"
                # },

                # {"type": "WEB", "seedUrls": [{ "url": "https://www.examplesite.com"}],
                #  "inclusionFilters": ["https://www\.examplesite\.com/.*\.html"],
                #  "exclusionFilters": ["https://www\.examplesite\.com/contact-us\.html"]
                # }
            ]
                
pp = pprint.PrettyPrinter(indent=2)

## Knowledge Base 생성하기

In [ ]:
knowledge_base = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}',
    kb_description=knowledge_base_description,
    data_sources=data_sources,
    chunking_strategy = "FIXED_SIZE", 
    suffix = f'{suffix}-f'
)

### 지식 베이스에 수집할 데이터 다운로드
다음과 같은 데이터를 사용할 것입니다:
- 첫 번째 데이터 소스로 로컬 디렉토리에 저장된 sythetic_dataset 폴더 내 자료

#### S3 Bucket data source로 데이터 업로드

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("../synthetic_dataset", data_bucket_name)

### 수집 작업 시작
KB(Knowledge Base)와 데이터 소스가 생성되면, 각 데이터 소스에 대한 수집 작업을 시작할 수 있습니다.
수집 작업 중에 KB는 다음과 같은 작업을 수행합니다:
- 데이터 소스에서 문서를 가져옴
- 텍스트를 추출하기 위해 전처리
- 제공된 청크 크기를 기반으로 문서를 분할
- 각 청크의 임베딩을 생성
- 생성된 임베딩을 벡터 데이터베이스(이 경우 OSS)에 저장

주의: 현재는 한 번에 하나의 수집 작업만 시작할 수 있습니다.

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
# keep the kb_id for invocation later in the invoke request
kb_id = knowledge_base.get_knowledge_base_id()
%store kb_id

### 2.2 Knowledge Base 테스트
이제 Knowledge Base를 사용할 수 있게 되었으므로 [retrieve](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html)와 [retrieve_and_generate](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) 함수를 사용하여 테스트해볼 수 있습니다.

#### Retrieve and Generate API를 사용한 Knowledge Base 테스트
먼저 retrieve and generate API를 사용하여 Knowledge Base를 테스트해보겠습니다. 이 API를 통해 Bedrock은 지식 베이스에서 필요한 참조를 검색하고 Bedrock의 기반 모델을 사용하여 최종 답변을 생성하는 작업을 처리합니다.

쿼리 = 
```
`Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019.`

'2019년 12월 31일로 종료되는 회계연도의 Octank Financial의 연결 현금흐름표에 대한 요약을 제공하시오.'
```

실제 QA 쌍에 따른 이 쿼리에 대한 올바른 응답은 다음과 같습니다:

```
The cash flow statement for Octank Financial in the year ended December 31, 2019 reveals the following:
- Cash generated from operating activities amounted to $710 million, which can be attributed to a $700 million profit and non-cash charges such as depreciation and amortization.
- Cash outflow from investing activities totaled $240 million, with major expenditures being the acquisition of property, plant, and equipment ($200 million) and marketable securities ($60 million), partially offset by the sale of property, plant, and equipment ($40 million) and maturing marketable securities ($20 million).
- Financing activities resulted in a cash inflow of $350 million, stemming from the issuance of common stock ($200 million) and long-term debt ($300 million), while common stock repurchases ($50 million) and long-term debt payments ($100 million) reduced the cash inflow.
Overall, Octank Financial experienced a net cash enhancement of $120 million in 2019, bringing their total cash and cash equivalents to $210 million.
```
 ------------------------------------------------------------------------------------------------------------------------------------------------------- 
```
2019년 12월 31일로 종료되는 연도의 Octank Financial 현금흐름표는 다음과 같습니다:
- 영업활동으로 인한 현금 창출액은 7억 1천만 달러로, 이는 7억 달러의 이익과 감가상각비 및 상각비와 같은 비현금성 비용에 기인합니다.
- 투자활동으로 인한 현금 유출액은 2억 4천만 달러로, 주요 지출은 유형자산 취득(2억 달러)과 시장성 있는 유가증권 취득(6천만 달러)이며, 이는 유형자산 매각(4천만 달러)과 만기 도래 유가증권(2천만 달러)으로 일부 상쇄되었습니다.
- 재무활동으로 인한 현금 유입액은 3억 5천만 달러로, 이는 보통주 발행(2억 달러)과 장기부채 발행(3억 달러)에서 발생했으며, 보통주 매입(5천만 달러)과 장기부채 상환(1억 달러)으로 현금 유입이 감소했습니다.
전반적으로 Octank Financial은 2019년에 1억 2천만 달러의 순현금 증가를 기록했으며, 이로 인해 총 현금 및 현금성자산이 2억 1천만 달러가 되었습니다.
```

In [ ]:
query = "2024년 기준 시도별 노령화지수를 비교했을 때, 가장 높은 지역과 가장 낮은 지역의 차이가 발생하는 원인을 인구구조와 사회경제적 특성을 고려하여 분석해줘"
#query = "Provide a summary of consolidated statements of cash flows of Octank Financial for the fiscal years ended December 31, 2019?"

In [ ]:
foundation_model = "anthropic.claude-3-haiku-20240307-v1:0"

response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": query
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

보시다시피, retrieve 및 generate API를 사용하면 최종 응답을 직접 받을 수 있으며, 이 응답을 생성하는 데 사용된 다양한 소스는 확인할 수 없습니다. 이제 retrieve API를 사용하여 Knowledge Base에서 소스 정보를 검색해 보겠습니다.

#### Retrieve API를 사용한 Knowledge Base 테스트
추가적인 제어 계층이 필요한 경우, retrieve API를 사용하여 쿼리와 가장 잘 일치하는 청크들을 검색할 수 있습니다. 이 설정에서는:

- 원하는 결과 수를 구성할 수 있습니다
- 자체 애플리케이션 로직으로 최종 답변을 제어할 수 있습니다
- API는 다음을 제공합니다:
    - 일치하는 콘텐츠
    - S3 위치
    - 유사도 점수
    - 청크 메타데이터

In [ ]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        #"text": "How many new positions were opened across Amazon's fulfillment and delivery network?"
        "text": "2024년 기준 시도별 노령화지수를 비교했을 때, 가장 높은 지역과 가장 낮은 지역의 차이가 발생하는 원인을 인구구조와 사회경제적 특성을 고려하여 분석해줘."
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

## 리소스 정리
모든 리소스를 삭제하기 위해 아래 섹션의 주석을 해제하고 실행하시기 바랍니다.

In [ ]:
# delete role and policies
print("===============================Deleting Knowledge Base and associated resources==============================\n")
# knowledge_base.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)